In [4]:
import whisper

In [5]:
model = whisper.load_model("base")

In [20]:
result = model.transcribe("audio_files/What_is_AI.mp3")

In [7]:
result["text"]

" What is AI? AI stands for artificial intelligence, and it's a way for computers to learn and work kind of like us. You can think of AI as a brain for a computer. Similar to how we use our brains to learn and make decisions, AI uses special computer programs to do the same thing. With AI, computers can help us with the stuff that would be really hard for people to do on their own. They can help doctors diagnose diseases, allow us to predict the weather more accurately, and even play video games. It works by using software to look at lots of information and find patterns. Just like how you can tell the difference between a dog and a cat, AI can learn to recognize things too. There are three main types of AI, supervised, unsupervised, and reinforcement learning. Each type learns in its own special way, but they all use lots of data to get better at what they do. AI is important because it helps us do things faster, more accurately, and sometimes even better than we could on our own. Plu

In [8]:
with open("documents/test_text.txt", "w") as text_file:
        text_file.write(result["text"])
        print("Transcription saved to test_text.txt")

Transcription saved to test_text.txt


In [9]:
import os
import warnings

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader

from dotenv import load_dotenv

# Ignore all warnings (not recommended in production code)
warnings.filterwarnings("ignore")

In [10]:
load_dotenv()

True

In [11]:
openai_key = os.getenv("OPENAI_API_KEY")
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [12]:
from langchain.text_splitter import TextSplitter
from langchain.document_loaders import TextLoader

In [13]:
transcript_loader = DirectoryLoader('./documents/', glob="./*.txt", loader_cls=TextLoader)
transcript_documents = transcript_loader.load()

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
text_chunks = text_splitter.split_documents(transcript_documents)

In [15]:
text_chunks

[Document(page_content='Dear Fellow Scholars, this is Two Minute Papers with Dr. KÃ¡roly Zsolnai-FehÃ©r. Today, we are going to have a look at an image creation AI called Perfusion that is completely out of this world. Imagine a text-to-image AI like Stable Diffusion or Meat Journey as a chef that can cook any dish. However, sometimes, we like the dish we get, but we would like to introduce our own ingredients to it. And normally, if we write a new recipe, a new text prompt, we get a completely new image, a new dish.', metadata={'source': 'documents\\NVIDIAs_New_AI_Text_To_Image_Supercharged.txt'}),
 Document(page_content="normally, if we write a new recipe, a new text prompt, we get a completely new image, a new dish. However, that is not what we want. What we want is the previous dish with a new ingredient. Let's have a look at an example. We have photos of our favorite teddy bear, and we would like it to create an image of it playing with a ball in the water. A previous technique do

In [16]:
## here we are using OpenAI embeddings but in future we will swap out to something else
embedding = OpenAIEmbeddings()
persist_directory = 'db'

# Batch chunk embedding + insert to DB
vectordb = Chroma.from_documents(documents=text_chunks, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)